### Назад в будущее: практическое руководство по путешествию во времени с Python

[Оригинал](https://proglib.io/p/nazad-v-budushchee-prakticheskoe-rukovodstvo-po-puteshestviyu-vo-vremeni-s-python-2019-12-01)

В Python есть несколько встроенных модулей для работы со временем и интервалами времени: `time`, `datatime`, `calendar`, `timeit`. Но когда какой модуль использовать? Рассмотрим на примерах.

Чтобы не обращаться далее к операции импорта, перечислим сразу все встроенные модули, рассматриваемые в этом руководстве:

In [42]:
import time
import timeit
import datetime
import calendar

### 1. Работа со шкалой времени: модуль time
#### 1.1. Понятие epoch
Работа с модулем [`time`](https://docs.python.org/3/library/time.html) в существенной мере зависит от используемой операционной системы. Время в библиотеке привязано к фиксированной начальной точке – эпохе (epoch). Узнаем эту начальную точку:

In [43]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

В Unix-системах точкой отсчета (epoch) является 1 января 1970 г. Функция `gmtime()` вернула объект именованного кортежа `struct_time`.

С помощью функции `time()` время, прошедшее с этой начальной точки, можно также вывести в секундах (seconds since the epoch):

In [44]:
time.time()

1583756116.7826967

Так как точка `epoch` для разных операционных систем может отличаться, число секунд, возвращаемое функцией `time()`, также может быть различным.

Время до точки `epoch` тоже существует, но значения секунд `seconds since the epoch` отрицательны. Если мы передадим функции `gmtime` отрицательное значение секунд, мы перенесемся в прошлое относительно момента времени `epoch`:

In [45]:
time.gmtime(-10**8)

time.struct_time(tm_year=1966, tm_mon=10, tm_mday=31, tm_hour=14, tm_min=13, tm_sec=20, tm_wday=0, tm_yday=304, tm_isdst=0)

#### 1.2. Секунды, struct_time и преобразование друг в друга

Итак, модуль `time` оперирует двумя основными типами объектов: `struct_time` и секундами с начала эпохи. Для взаимных преобразований используются следующие функции:

1. `gmtime()`: из секунд в struct_time для UTC.
2. `localtime()`: из секунд в `struct_time` для местного времени.
3. `calendar.timegm()` (не модуль time): из `struct_time` для UTC в секунды.
4. `mktime()`: из `struct_time` местного времени в секунды.

UTC – стандартное обозначение [всемирного координированного времени](https://ru.wikipedia.org/wiki/%D0%92%D1%81%D0%B5%D0%BC%D0%B8%D1%80%D0%BD%D0%BE%D0%B5_%D0%BA%D0%BE%D0%BE%D1%80%D0%B4%D0%B8%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%BE%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F) без учета часовых поясов. Начало отсчета `epoch` привязано к UTC, то есть не зависит от местного времени. Поэтому UTC удобнее использовать для общения с внешним миром.

In [46]:
time.gmtime(1574869797)

time.struct_time(tm_year=2019, tm_mon=11, tm_mday=27, tm_hour=15, tm_min=49, tm_sec=57, tm_wday=2, tm_yday=331, tm_isdst=0)

In [47]:
time.localtime(1574869797)  # заметьте отличие в выводе значения tm_hour

time.struct_time(tm_year=2019, tm_mon=11, tm_mday=27, tm_hour=18, tm_min=49, tm_sec=57, tm_wday=2, tm_yday=331, tm_isdst=0)

В отсутствии аргумента функции `gmtime()` и `localtime()` возвращают значение для текущего времени – соответственно UTC и местное время.

Для преобразования объекта `struct_time` в секунды можно или передать сам объект, или кортеж целых чисел. Порядок элементов в кортеже:

1. Год `tm_year`
2. Месяц `tm_mon` – целое число (1 – Январь, 12 – Декабрь)
3. День месяца `tm_day`
4. Час `tm_hour` – целое число в диапазоне от 0 до 23
5. Минута `tm_min`
6. Секунда `tm_sec`
7. День недели `tm_wday` – целое число от 0 (Понедельник) до 6 (Воскресенье)
8. День года `tm_yday`
9. Целочисленный флаг `tm_isdst` для учета перехода на летнее время (daylight saving time, DST): 1 – переход на летнее время учитывается, 0 – не учитывается, -1 – неизвестно.

In [48]:
time.mktime((2015, 10, 21, 7, 28, 0, 2, 294, -1))

1445401680.0

Очевидно, что составлять такой кортеж вручную – задача неблагодарная, ведь нужно знать и день недели, и номер дня в году. Обычно используются «готовые» объекты (для «ручного» формирования дат удобнее применять описанный далее модуль `datetime`):

In [49]:
time.mktime(time.localtime())

1583756117.0

Для перевода не местного, а международного времени в секунды необходимо использовать модуль `calendar`:

In [50]:
calendar.timegm(time.gmtime())

1583756117

Заметим, что, в отличие от представления в виде секунд, `struct_time` не хранит составляющих времени, меньших, чем секунда.

Удобство использования `struct_time` заключается в том, что это именованный кортеж. Значит, можно писать более ясный код: вместо индексов элементы объекта вызываются по ключам с говорящими именами:

In [51]:
t = time.localtime()
f'Итак, на дворе {t.tm_year}-й год.'

'Итак, на дворе 2020-й год.'

Кроме вышеперечисленных параметров-меток, `struct_time` содержит скрытые. Так, местным законодательством каждой страны регулируется переход на летнее время. Узнать, действует ли сейчас летнее время, можно следующим образом (в России в 2014 году произведен переход на постоянное «зимнее» время):

In [52]:
t.tm_isdst

0

Считаем часовой пояс:

In [53]:
t.tm_zone

'MSK'

Смещение местного времени относительно UTC в секундах:

In [54]:
t.tm_gmtoff

10800

#### 1.3. Строки временных меток

Распространенная задача – преобразование объектов указанных типов в строки вида timestamp, например, `Mon Dec 2 18:30:20 2019`. Для этого применяются функции `ctime()` и `asctime()`:

- `ctime()` – принимает время в секундах
- `asctime()` – принимает `struct_time` (по умолчанию используется местное время)

In [55]:
time.ctime(time.time())  # преобразует время в секундах в timestamp для местного времени

'Mon Mar  9 15:15:17 2020'

In [56]:
time.asctime()  # Аналогично time.asctime(time.localtime())

'Mon Mar  9 15:15:18 2020'

In [57]:
time.asctime(time.gmtime())  # Время UTC

'Mon Mar  9 12:15:18 2020'

Хотя строковый вывод функций `ctime()` и `asctime()` довольно удобен, может потребоваться альтернативный формат. Для гибкого форматирования в библиотеку `time` включена функция `strftime()`. Функция принимает строку шаблона форматирования со спецификаторами и сам объект времени.

In [58]:
time.strftime('%d.%m.%Y', time.localtime())

'09.03.2020'

Функция strftime() также удобна для автоматической локализации строк:

In [59]:
import locale

In [60]:
locale.setlocale(locale.LC_TIME, 'ru_RU.utf8')

'ru_RU.utf8'

In [61]:
time.strftime('Текущее время: %c', time.localtime())

'Текущее время: Пн 09 мар 2020 15:15:18'

Список спецификаторов шаблона:

- `%a`, `%A` – аббревиатура и полное название дня недели (Чт, Четверг)
- `%b`, `%B` – то же для месяца с учетом склонения (ноя, ноября)
- `%с` – локализованная строка временной метки
- `%d` – день месяца (28)
- `%H`, `%I` – Час в 24- и 12-часовом представлении (17, 05)
- `%j` – номер дня года (в представлении от 001 до 366)
- `%m` – двузначное представление месяца (от 01 до 12)
- `%M` – двузначное представление минут (от 00 до 59)
- `%p` – местный эквивалент AM и PM
- `%S` – двузначное представление секунд
- `%W` – двузначное представление номера недели, первый день – Пн (%U для Вс)
- `%w` – двузначное представление номера дня недели
- `%x`, `%X` – принятый способ представления даты и времени.
- `%y`, `%Y` – двузначное (без века) и четырехзначное представление года
- `%z`, `%Z` – обозначение часового пояса в четырехзначном формате со знаком плюс или минус и в виде названия часового пояса

Пример одновременного использования нескольких спецификаторов:

In [62]:
s = """Сегодня %A, %d %B. В России эту дату обычно записывают
следующим образом: %x или сокращенно: %d.%m.%y.
Это %j день года, %W неделя. На часах %X.
Часовой пояс: %Z."""

print(time.strftime(s, time.localtime()))

Сегодня Понедельник, 09 Март. В России эту дату обычно записывают
следующим образом: 09.03.2020 или сокращенно: 09.03.20.
Это 069 день года, 10 неделя. На часах 15:15:18.
Часовой пояс: MSK.


Что, если у нас есть строка, содержащая метку времени, а мы хотим распарсить ее в объект `struct_time`, чтобы обработать его в Python? Для этого есть функция `strptime()`. Первый аргумент – строка, второй – правило, описанное через те же спецификаторы:

In [63]:
time.strptime('Окт 21 2015 07:28', '%b %d %Y %H:%M')

time.struct_time(tm_year=2015, tm_mon=10, tm_mday=21, tm_hour=7, tm_min=28, tm_sec=0, tm_wday=2, tm_yday=294, tm_isdst=-1)

Функция `strptime()` позволяет кратко задавать `struct_time`, не используя все девять позиций кортежа. Неизвестные элементы вычисляются или на их место подставляются значения по умолчанию.

#### 1.4. Приостановка выполнения кода и оценка производительности

Одна из наиболее часто используемых функций модуля `time` – функция `sleep()`, выполняющая задержку исполнения программного кода на переданное число секунд (можно использовать дробные значения):

In [64]:
print(time.strftime('Текущее время: %X.'))
print('Задержка...')
time.sleep(5)
print('Прошло время.')
print(time.strftime('Текущее время: %X.'))

Текущее время: 15:15:18.
Задержка...
Прошло время.
Текущее время: 15:15:23.


Функция `sleep()` нередко используется для тестирования кода, намеренного внесения задержек на различных этапах выполнения программы.

Для оценки производительности однократно запускаемых команд применяется функция `perf_counter()`, обеспечивающая лучшее разрешение по времени на коротких интервалах:

In [87]:
def longrunning_function():
    for i in range(3):
        time.sleep(1)
        
def shortrunning_function():
    n = 1
    for i in range(2, 100):
        n *= i

start = time.perf_counter()
longrunning_function()
end = time.perf_counter()
print(f"Выполнение longrunning_function() заняло {end-start} c.")

start = time.perf_counter()
shortrunning_function()
end = time.perf_counter()
print(f"Выполнение shortrunning_function() заняло {end-start} c.")

Выполнение longrunning_function() заняло 3.0025332409995826 c.
Выполнение shortrunning_function() заняло 0.0002509559999452904 c.


В Python версии 3.7 добавлена функция `perf_counter_ns()` – работает так же, но длительность выводится в наносекундах, что удобнее для совсем малых интервалов времени и быстро исполняемых команд.

Более удобные методы для измерения производительности фрагмента кода предоставляет модуль `timeit`.

### 2. Оценка производительности: timeit

В момент запуска программы в фоновом режиме также запускается множество сторонних процессов. Модуль [`timeit`](https://docs.python.org/3.8/library/timeit.html) за счет многократного запуска фрагмента нивелирует неоднородность длительности его выполнения.

У модуля `timeit` есть интерфейс командной строки и интерфейс для вызова в коде. Во втором случае выводится время в секундах, которое длится общее количество запусков. Так как значение number по умолчанию составляет 1 млн повторений, можно считать, что при дефолтном запуске выводится среднее время операции в микросекундах. При вызове timeit в командной строке достаточное количество повторений определяется автоматически.

Сравним скорость выполнения операция конкатенации при использовании генератора и функции `map()`:

```python
python3 -m timeit '"-".join(str(n) for n in range(100))'
100000 loops, best of 3: 14.6 usec per loop

python3 -m timeit '"-".join(map(str, range(100)))'
100000 loops, best of 3: 9.72 usec per loop
```

Сравним с вызовом через интерпретатор Python:

In [66]:
timeit.timeit('"-".join(str(n) for n in range(100))')

18.12641012400036

In [67]:
timeit.timeit('"-".join(map(str, range(100)))')

12.482238461000634

Кроме куска кода, функции `timeit()` можно передать строку setup, однократно выполняемую перед началом повторения кода `stmt`. В `setup`, например, можно вынести импорт библиотек:

In [68]:
mysetup = 'from math import sqrt'

mycode = '''
mylist = []
for i in range(100):
    mylist.append(sqrt(i))
'''
        
timeit.timeit(stmt = mycode,
              setup = mysetup,
              number = 10000)

0.11019666099946335

В блокнотах Jupyter команда `timeit` относится к числу магических. С одним знаком процента она действует в пределах строки кода, с двумя – в границах ячейки:

In [69]:
%timeit s = "-".join(str(n) for n in range(100))

18.4 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [70]:
%%timeit
mylist = []
for i in range(100):
    mylist.append(i**0.5)

16.6 µs ± 2.03 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Магические команды `%time` и `%%time` делают те же операции без многократного повторения. Это приводит к завышенным результатам, но позволяет быстрее получить оценку производительности:

In [71]:
%time s = "-".join(str(n) for n in range(100))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 27.2 µs


In [72]:
%%time
mylist = []
for i in range(100):
    mylist.append(i**0.5)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 68.2 µs


### 3. Работа с датами: datetime

Вернемся к вопросу перемещения во времени. Модуль [`datetime`](https://docs.python.org/3/library/datetime.html) поддерживает различные операции для работы с датами, например, определение интервала между двумя днями.

Структура представления времени в `datetime` похожа на `struct_time` в модуле `time`:

In [83]:
t = datetime.datetime.now()
t

datetime.datetime(2020, 3, 9, 15, 17, 22, 500553)

In [86]:
print(t)

2020-03-09 15:17:22.500553


Выведем отдельно дату и время:

In [85]:
f'Сегодня {t.date()}. Время: {t.time()}.'

'Сегодня 2020-03-09. Время: 15:17:22.500553.'

Аналогично извлекаются год, месяц и т.д.:

In [76]:
f"Год {t.year}, месяц {t.month}, день {t.day}, {t.hour} ч. {t.minute} мин. {t.second} сек."

'Год 2020, месяц 3, день 9, 15 ч. 16 мин. 14 сек.'

Модуль datetime также удобен для «ручного» задания дат и автоматизации арифметических операций с датами. Узнаем интервал времени между двумя главными датами сюжета фильма «Назад в будущее 2»:

In [77]:
today = datetime.datetime(year=1985, month=10, day=26, hour=21, minute=0)
future = datetime.datetime(year=2015, month=10, day=21, hour=19, minute=28)
delta = future - today
print(delta)

10951 days, 22:28:00


Добавление найденной разности к первой дате «возвращает» нас в «будущее»:



In [78]:
print(today + delta)

2015-10-21 19:28:00


Узнаем, какое число будет через четыре недели. Для форматирования строк в модуле `datetime` имеется функция `strftime()` с теми же спецификаторами, что и в модуле `time`:

In [79]:
today = datetime.datetime.now()
future = today + datetime.timedelta(days=28)
f = '%d.%m.%y'
print(today.strftime('Сегодня: ' + f))
print(future.strftime('Через 28 дней будет: ' + f))

Сегодня: 09.03.20
Через 28 дней будет: 06.04.20


Если вам важнее оперировать не датами, а неделями, днями недели, месяцами, годами, то вам нужен модуль `calendar`.

### 4. Работа с календарем: calendar

Модуль [`calendar`](https://docs.python.org/3.8/library/calendar.html) содержит функции для работы с календарем. В частности, умеет генерировать строки и HTML для вывода каленадарей месяцев и годов. Для наглядности напечатаем календарь на декабрь 2019 года:

In [80]:
calendar.prmonth(2019, 12)

    Декабрь 2019
Пн Вт Ср Чт Пт Сб Вс
                   1
 2  3  4  5  6  7  8
 9 10 11 12 13 14 15
16 17 18 19 20 21 22
23 24 25 26 27 28 29
30 31


При помощи calendar можно не только «рисовать» календари, но и осуществлять итерации по их составляющим.

В качестве примера рассмотрим следующую практическую задачу. Во многих музеях существует один день месяца, когда посещение музея для всех лиц или отдельных категорий граждан происходит без взимания платы. Например, в Эрмитаже это третий четверг месяца. Вычислим даты бесплатных дней посещения Эрмитажа на 2020 год:

In [81]:
free_days = []

for i in range(1, 13):
    c = calendar.monthcalendar(2020, i)
    first_week = c[0]
    third_week = c[2]
    fourth_week = c[3]

    # Если на первой неделе месяца есть четверг, то третий
    # четверг должен быть на третьей неделе. Если нет, то
    # на четвертой
    if first_week[calendar.THURSDAY]:
        free_day = third_week[calendar.THURSDAY]
    else:
        free_day = fourth_week[calendar.THURSDAY]
    s = '{0} {1}'.format(free_day, calendar.month_name[i])
    free_days.append(s)

print(", ".join(free_days))

16 Январь, 20 Февраль, 19 Март, 16 Апрель, 21 Май, 18 Июнь, 16 Июль, 20 Август, 17 Сентябрь, 15 Октябрь, 19 Ноябрь, 17 Декабрь


### 5. Сторонние библиотеки

[Arrow: Better dates & times for Python](https://arrow.readthedocs.io/en/latest/)